## Modelo de aprendizado não-supervisionado
- Esse modelo é usado para encontrar padrões nos dados.

In [0]:
from sklearn import datasets
import numpy as np
import pandas as pd 

# Load the iris dataset
iris_bunch = datasets.load_iris()

# Convert to DataFrame
iris = pd.DataFrame(data=iris_bunch.data, columns=iris_bunch.feature_names)
iris['species'] = iris_bunch.target

iris_df = iris.drop(columns='species')
# Print the 'species' column
display(iris)

In [0]:
from sklearn.cluster import KMeans

model = KMeans(n_clusters=3)
model.fit(iris_df)
rotulo = model.predict(iris_df) # fit_predict() é o mesmo que usar as duas funções

centroid = model.cluster_centers_

print(rotulo)

In [0]:
import matplotlib.pyplot as plt 

plt.figure(figsize=(10,5))
x = iris_df['sepal length (cm)']
y = iris_df['petal length (cm)']
plt.scatter(x,y,c=rotulo)

plt.scatter(centroid[:,0],centroid[:,1],color='red',marker='D',s=100)

plt.show()

### Avaliando a qualidade de um cluster
- Nesse caso, é possível comparar os conjuntos com as espécies de íris.
Entretanto, na maioria dos casos, não terá rótulos nos datasets.
- É necessário uma medida que utilize apenas os agrupamentos e as próprias amostras. --> **Inércia**

- Inércia = A distância entre as amostras e seus centróides.

In [0]:
df = pd.DataFrame({'labels':rotulo,'species':iris['species']})

ct = pd.crosstab(df['labels'], df['species'])
print(ct)

In [0]:
print(model.inertia_)

In [0]:
# Determinando a melhor inérica

k = range(1,6)
inercia = []

for i in k:
    model_c = KMeans(n_clusters=i)
    model_c.fit(iris_df)
    inercia.append(model_c.inertia_)

plt.figure(figsize=(10,5))
plt.plot(k,inercia,marker='o', color='purple')
plt.xlabel('número de clusters')
plt.ylabel('inércia')
plt.show()

### Transformado features para melhor clusterização
- Em alguns conjuntos de dados, os clusters não irão corresponder tão bem.
- Isso pode acontecer se as features tiverem variâncias muito diferentes, como acontece com o dataset de vinhos do Piemonte.

- A variância da feature corresponde à influência no algoritmo.

In [0]:
from sklearn.datasets import load_wine
import pandas as pd

# Carregar o conjunto de dados dos vinhos
wine_data = load_wine()
wine_df = pd.DataFrame(data=wine_data.data, columns=wine_data.feature_names)

print(wine_df.var())

In [0]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler() 
scaler.fit(wine_df)
StandardScaler(copy=True,with_mean=True, with_std=True)
amostra_normalizada = scaler.transform(wine_df)

In [0]:
# Também é possível fazer o mesmo usando pipelines
from sklearn.pipeline import make_pipeline
from sklearn.cluster import KMeans

# Assuming scaler and wine_df are defined somewhere in your code
scaler = StandardScaler()
kmeans = KMeans(n_clusters=3)
pipeline = make_pipeline(scaler, kmeans)
pipeline.fit(wine_df)

rotulo = pipeline.predict(wine_df)

# Add the predicted labels to the DataFrame
df = wine_df.copy()
df['rotulo'] = rotulo

# Create the crosstab
ct = pd.crosstab(df['rotulo'], wine_data.target)
display(ct)

## T-SNE e Hierarchical Clustering

####Visualizando hierarquias:
**Agrupamento Hierárquico:**
- Grupos podem formar uma hierarquia, estando contidos uns nos outros.
- É possível organizar qualquer tipo de dado em uma hierarquia.
- Etapas:
1. Cada grupo é separado em clusters.
2. A cada passo, os 2 clusters mais próximos são mesclados.
3. Continua mesclando até todos os grupos estarem juntos em um único cluster.

- T-SNE = Cria um mapa 2D de um conjunto de dados e transmite informações sobre a proximidade das amostras umas das outras.


In [0]:
import matplotlib.pyplot as plt 
from scipy.cluster.hierarchy import linkage, dendrogram
import pandas as pd

# Carregar os dados
df = pd.read_csv('https://assets.datacamp.com/production/repositories/655/datasets/2a1f3ab7bcc76eef1b8e1eb29afbd54c4ebf86f2/eurovision-2016.csv')

# Agrupar por país e tirar a média dos votos (um país por linha)
df_grouped = df.groupby("To country")[["Jury A", "Jury B", "Jury C", "Jury D", "Jury E", "Jury Rank", "Televote Rank"]].mean()

# Aplicar o linkage
mescla = linkage(df_grouped, method='complete')

# Plotar o dendrograma
plt.figure(figsize=(14,5))
dendrogram(mescla, 
           labels=df_grouped.index.tolist(), 
           leaf_rotation=90, 
           leaf_font_size=12,
           color_threshold=8
           )
plt.show()

### Agrupamento hierárquico
- É possível extrair cluster de estágios intermediários de um clustering hierárquico.
- O estágio intermediário é especificado pela escolha de altura no dendograma.
--> O eixo y é a distância entre os clusters mesclados.

- A função fcluster() retorna um array com rótulos de cluster

In [0]:
from scipy.cluster.hierarchy import linkage, fcluster

mescla = linkage(df_grouped, method='complete')

rotulo = fcluster(mescla, 8, criterion='distance')

print(rotulo)

In [0]:
import pandas as pd 

pares = pd.DataFrame({'rotulo': rotulo, 'paises':df_grouped.index})
print(pares.sort_values('rotulo')) 

### T-SNE 
- Siginifica **incorporação estocástica de vizinho distribuída em t**
- Mapeia amostras de seu espaço de alta dimensão em um espaço bidimensional/tridimensional para que possam ser visualizadas.

- A função fit_transform() faz o ajuste e a transformação.
- Não possui métodos fit() e transform() separados.

In [0]:
import matplotlib.pyplot as plt 
from sklearn.manifold import TSNE

amostra = iris.drop('species',axis=1)
especies = iris['species']

modelo = TSNE(learning_rate=100)
transformado = modelo.fit_transform(amostra)

xs = transformado[:,0]
ys = transformado[:,1]

plt.figure(figsize=(10,5))
plt.scatter(xs,ys,c=especies)
plt.show()


### Redução de dimensionalidade (PCA)
- PCA = Análise de Componentes Principais.
- É constantemente usado antes do aprendizado supervisionado para melhorar o desempenho e generalização do modelo.
 
- Pode ser usado para descartar features que geram ruídos.

O PCA reduz a dimensionalidade em 2 etapas:
1. Descorrelação:   
--> Rotaciona as amostras mantendo elas alinhadas com os eixos de coordenadas.
--> Desloca as amostrar para que a média seja 0.
--> A informação não é perdida.
2. Seleção de componentes principais
--> Encontrar os componentes de maior variância (principais)
--> 

In [0]:
from sklearn.decomposition import PCA

modelo = PCA()
modelo.fit(amostra)

transformado = modelo.transform(amostra)
print(modelo.components_)

### Dimensão intrínseca
- É o número de características necessárias para aproximar o dataset. --> Features do PCA com variância significativa.
- Diz o quanto um dataset pode ser compactado.
- Quanto maior, mais informativo. As features de menor variância podem ser descartadas.

.n_components_  
.explained_variace_  


In [0]:
import matplotlib.pyplot as plt 
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

amostra = iris.drop('species',axis=1)

scaler = StandardScaler()
pca = PCA()
pipeline = make_pipeline(scaler, pca)

pipeline.fit_transform(amostra)

features = range(pca.n_components_)

plt.bar(features, pca.explained_variance_)
plt.xticks(features)
plt.ylabel("Variância")
plt.xlabel("Features")
plt.show()

#  dimensão intrínseca mais provável, com base neste gráfico, seria 2

In [0]:
#n_components = Quantos recursos o modelo deve manter.
# Se possível, escolher a dimensão intrínseca
especies = iris['species']

pca = PCA(n_components=2)
pca.fit(amostra)

transformado = pca.transform(amostra)

xs = transformado[:,0]
ys = transformado[:,1]

plt.scatter(xs,ys,c=especies)
plt.show()

### NMF
- NMF = Fatoração de Matrizes Não Negativas.
- É uma técnica de redução de dimensionalidade

- Decompõe amostras como somas de suas partes.